In [1]:
import pyspark
import pyspark.sql.functions as F
import time, shutil, os
import pandas as pd

In [2]:

cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
os.chdir(parent_dir)

In [ ]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = (
    SparkSession.builder 
    .master("local[6]")
    .appName("delta")
    .config("spark.driver.memory", "13g")
    .config("spark.jars.packages",
            "io.delta:delta-core_2.12:3.3.2")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

25/06/17 22:39:32 WARN Utils: Your hostname, quero-Inspiron-15-3520 resolves to a loopback address: 127.0.1.1; using 192.168.15.151 instead (on interface wlp2s0)
25/06/17 22:39:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/eric-yoshida/pyspark-env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/eric-yoshida/.ivy2/cache
The jars for the packages stored in: /home/eric-yoshida/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-aa5f8d0f-d924-400f-b373-aee7de46f216;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.2 in central
	found io.delta#delta-storage;3.3.2 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 121ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.2 from central in [default]
	io.delta#delta-storage;3.3.2 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     | 

In [4]:
data_path = "./data/processed/"

united_df = spark.read.json(data_path + "united_df.json")

In [5]:
united_df.limit(10).toPandas()

,asin,image,overall,reviewText,reviewTime,reviewerID,reviewerName,style,summary,unixReviewTime,verified,vote
0,B000FA5KK0,None,4.0,"pretty good story, a little exaggerated, but I...","07 3, 2014",A2LSKD2H9U8N0J,sandra sue marsolek,Kindle Edition,pretty good story,1404345600,True,None
1,B000FA5KK0,None,5.0,"If you've read other max brand westerns, you k...","05 26, 2014",A2QP13XTJND1QS,Tpl,Kindle Edition,A very good book,1401062400,True,None
2,B000FA5KK0,None,5.0,"Love Max, always a fun twist","09 16, 2016",A8WQ7MAG3HFOZ,Alverne F. Anderson,Kindle Edition,Five Stars,1473984000,True,None
3,B000FA5KK0,None,5.0,"As usual for him, a good book","03 3, 2016",A1E0MODSRYP7O,Jeff,Kindle Edition,a good,1456963200,True,None
4,B000FA5KK0,None,5.0,MB is one of the original western writers and ...,"09 10, 2015",AYUTCGVSM1H7T,DEHS - EddyRapcon,Kindle Edition,A Western,1441843200,True,2
5,B000FA5KK0,None,5.0,great book,"06 14, 2015",A9DXGM6YGV14D,Linda DeWachter,Kindle Edition,Five Stars,1434240000,True,None
6,B000FA5KK0,None,3.0,"A good, solid Western - yes, a little contrive...","06 2, 2015",A3MF8G33UKQLGL,mzbeastle,Kindle Edition,"Fast read, entertaining, like a 1930s Western ...",1433203200,True,3
7,B000FA5KK0,None,4.0,ALMOST BEEN TOO LONG SINCE I READ IT. GOOD REA...,"05 3, 2015",A12AIK6DSUF1EW,R. DAVIS,Kindle Edition,ANOTHER GOOD WESTERN,1430611200,True,None
8,B000FA5KK0,None,5.0,Enjoyed this book and will read more from this...,"02 2, 2015",A2RW1CXT3XSLXW,Barbara Rigsby,Kindle Edition,Good,1422835200,True,None
9,B000FA5KK0,None,4.0,A very good read for you Western fans. Great ...,"12 19, 2014",A2ID7H6SRK5XS0,Richard Johnson,Kindle Edition,Get it,1418947200,True,None


In [6]:
delta_path = "./data/delta/united_data"

start_time = time.time()

united_df.write.format("delta").mode("overwrite").option("overwriteSchema", "True").save(delta_path)

time_taken = time.time() - start_time

print(f"Time taken to write Delta table: {time_taken:.2f} seconds")

25/06/17 22:40:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Time taken to write Delta table: 23.49 seconds


In [7]:
def get_folder_size(path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if os.path.isfile(fp):
                total_size += os.path.getsize(fp)
    return total_size

In [8]:
folder_size_bytes = get_folder_size(delta_path)
folder_size_mb = folder_size_bytes / (1024 * 1024)

print(f"Tamanho real da tabela Delta: {folder_size_mb:.2f} MB")

Tamanho real da tabela Delta: 1819.30 MB


In [9]:
from pyspark.sql import Row

data = [
    Row(
        experiment_name="delta_1",
        file_size=folder_size_mb,
        time_taken=time_taken
    )
]

experiment_df = spark.createDataFrame(data)
experiment_df.write.mode("append").json("./data/processed/experiments.json")